In [ ]:
# Run on gpu5 for memory

# conda activate anndata

import os
import sys
import pandas as pd
import anndata as ad
import concurrent.futures

# sys.path.append("/mnt/lareaulab/reliscu/code")

from junction2psi import *

os.chdir("/mnt/lareaulab/reliscu/projects/NSF_GRFP/data/scRNA-seq/BICCN/mouse_ACA")

Here I create pseudobulk splice junction count data, using the same cells in each sample as were used to generate the gene-level pseudobulk. These pseudobulk SJ counts will be used to calculate pseudobulk PSIs for each exon.

In [ ]:
dtype = np.float64
intron_file = "/mnt/lareaulab/reliscu/data/GENCODE/GRCm39/psix_annotation/intron_file.tab.gz"
sj_dir = "/mnt/lareaulab/reliscu/projects/NSF_GRFP/data/scRNA-seq/tasic_2018/ALM/processed/STAR"
sdata = ad.read_h5ad("/mnt/lareaulab/reliscu/projects/NSF_GRFP/data/scRNA-seq/tasic_2018/ALM/tasic_2018_ALM_STAR_SJ_counts.h5ad")
pseudobulk_meta = pd.read_csv("/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/data/SyntheticDatasets/SyntheticDataset1_20pcntCells_25SD_200samples_legend_03-56-13.csv")

In [183]:
intron_table = read_intron_file(intron_file)

sj_name = []
sj_counts = []
with open(f"{sj_dir}/SJ.out.tab", "rb") as fh:
    for line in fh:
        sj_name, sj_counts = process_SJ_line2(line, sj_name, sj_counts)
sj_table = pd.DataFrame({"Counts": sj_counts}, index=sj_name)
sj_table = sj_table.astype(dtype)

sj_table.index = sj_table.index.astype(str)
intron_table['intron'] = intron_table['intron'].astype(str)
sj_table = pd.merge(intron_table, sj_table, # Subset to SJs in annotated intron table
                    left_on="intron", how="left", 
                    right_index=True).fillna(0)

# Save original row order
sdata.var['Row'] = np.arange(sdata.var.shape[0])
sj_merge = pd.merge(sdata.var, sj_table,
                    left_index=True, right_on="intron", how="inner")

In [191]:
sj_merge.index.is_unique

True

In [194]:
sdata_filt.shape

(9573, 136261)

In [ ]:
# Subset sdata to annotated exons
sdata_filt = sdata[:,sj_merge['Row'].astype(int).tolist()].copy()
# Update metadata
sdata_filt.var = sj_merge

/mnt/lareaulab/reliscu/anaconda3/envs/anndata/lib/python3.12/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [ ]:
count_mat = pseudobulk_meta.iloc[:, 2].astype(int).to_numpy() # shape: cells × n_pseudobulks
labels = pseudobulk_meta['Cell.name'].astype(str).tolist()

pseudo_list = []
for i in range(count_mat.shape[1]):
    counts = count_mat[:,i]
    rep_labels = np.repeat(labels, counts)
    
    idx = sdata.obs_names.get_indexer(rep_labels)
    sub = sdata[idx,:]
    pseudo_list.append(sub.sum(axis=0))

pseudo_array = np.vstack(pseudo_list)

(1, 1826616)

In [234]:
M = sdata_sample.layers.get('counts', sdata_sample.X)

In [238]:
M.sum(axis=0)

matrix([[0, 0, 0, ..., 0, 0, 0]])

In [ ]:
sdata[sdata.obs.index ]

,sample_name,title,source_name,organism,donor_id,donor_sex,donor_genotype,injection_type,injection_target,injected_material,...,rna_amplification_set,sequencing_tube,sequencing_batch,sequencing_qc_pass_fail,cell_class,cell_subclass,cell_cluster,molecule,GEO_Sample,GEO_Sample_Title
SRR7311317,F2S4_160830_001_A01,F2S4_160830_001_A01,Anterior Lateral Motor Cortex (ALM),Mus musculus,266796,M,Ai14(RCL-tdT)/Ai14(RCL-tdT),Retrograde,SSs,RV-dGL-Cre,...,A8S4_160901_04,LS-15546,R8S4-160923,Pass,Glutamatergic,L2/3 IT,L2/3 IT ALM Sla,polyA RNA,GSM3189842,ALM_Ai14(RCL-tdT)/Ai14(RCL-tdT)_Retrograde_1
SRR7311318,F2S4_160830_002_B01,F2S4_160830_002_B01,Anterior Lateral Motor Cortex (ALM),Mus musculus,266796,M,Ai14(RCL-tdT)/Ai14(RCL-tdT),Retrograde,SSs,RV-dGL-Cre,...,A8S4_160901_04,LS-15546,R8S4-160923,Pass,Glutamatergic,L6 IT,L6 IT ALM Oprk1,polyA RNA,GSM3189842,ALM_Ai14(RCL-tdT)/Ai14(RCL-tdT)_Retrograde_1
SRR7311319,F2S4_160902_002_F01,F2S4_160902_002_F01,Anterior Lateral Motor Cortex (ALM),Mus musculus,266799,F,Ai14(RCL-tdT)/Ai14(RCL-tdT),Retrograde,ZI,RV-dGL-Cre,...,A8S4_160907_03,LS-15552,R8S4-160923,Pass,Glutamatergic,L5 PT,L5 PT ALM Npsr1,polyA RNA,GSM3189842,ALM_Ai14(RCL-tdT)/Ai14(RCL-tdT)_Retrograde_1
SRR7311320,F2S4_160902_002_G01,F2S4_160902_002_G01,Anterior Lateral Motor Cortex (ALM),Mus musculus,266799,F,Ai14(RCL-tdT)/Ai14(RCL-tdT),Retrograde,ZI,RV-dGL-Cre,...,A8S4_160907_03,LS-15552,R8S4-160923,Pass,Glutamatergic,L6 CT,L6 CT ALM Cpa6,polyA RNA,GSM3189842,ALM_Ai14(RCL-tdT)/Ai14(RCL-tdT)_Retrograde_1
SRR7311322,F2S4_160831_004_A01,F2S4_160831_004_A01,Anterior Lateral Motor Cortex (ALM),Mus musculus,266797,M,Ai14(RCL-tdT)/Ai14(RCL-tdT),Retrograde,ALM-c,RV-dGL-Cre,...,A8S4_160907_04,LS-15553,R8S4-160923,Pass,Glutamatergic,L5 IT,L5 IT ALM Npw,polyA RNA,GSM3189842,ALM_Ai14(RCL-tdT)/Ai14(RCL-tdT)_Retrograde_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR7321380,FJS4_170405_005_G01,FJS4_170405_005_G01,Anterior Lateral Motor Cortex (ALM),Mus musculus,ANM382288,F,wt/wt,Retrograde,VAL,AAV-SL1-CAG-GFP,...,A8S4_170428_02,SM-D9E61,R8S4-170511,Pass,Glutamatergic,L5 PT,L5 PT ALM Hpgd,polyA RNA,GSM3189894,ALM_wt/wt_Retrograde_1
SRR7321381,FJS4_170405_006_A01,FJS4_170405_006_A01,Anterior Lateral Motor Cortex (ALM),Mus musculus,ANM382288,F,wt/wt,Retrograde,VAL,AAV-SL1-CAG-GFP,...,A8S4_170428_02,SM-D9E61,R8S4-170511,Pass,Glutamatergic,L5 PT,L5 PT ALM Hpgd,polyA RNA,GSM3189894,ALM_wt/wt_Retrograde_1
SRR7321382,FJS4_170405_006_B01,FJS4_170405_006_B01,Anterior Lateral Motor Cortex (ALM),Mus musculus,ANM382288,F,wt/wt,Retrograde,VAL,AAV-SL1-CAG-GFP,...,A8S4_170428_02,SM-D9E61,R8S4-170511,Pass,Glutamatergic,L5 PT,L5 PT ALM Hpgd,polyA RNA,GSM3189894,ALM_wt/wt_Retrograde_1
SRR7321383,FJS4_170405_006_C01,FJS4_170405_006_C01,Anterior Lateral Motor Cortex (ALM),Mus musculus,ANM382288,F,wt/wt,Retrograde,VAL,AAV-SL1-CAG-GFP,...,A8S4_170428_02,SM-D9E61,R8S4-170511,Pass,Glutamatergic,L5 PT,L5 PT ALM Hpgd,polyA RNA,GSM3189894,ALM_wt/wt_Retrograde_1


In [ ]:
def sum_sj_counts(mask):
    cell_subset = sj_counts # Get cells used to make each pseudobulk sample
    pseudo_sample = pd.concat(cell_subset, axis=1).sum(axis=1) # Sum SJ counts across cells
    return pseudo_sample

In [ ]:
# Sum SJ reads to make pseudobulk samples

cell_incl_status = pseudobulk_meta.iloc[:, 2:].T.values.astype(bool) # Each pseudobulk sample is comprised of a different subset of cells

pseudo_list = []
with concurrent.futures.ThreadPoolExecutor() as executor:
     pseudo_list = list(executor.map(sum_sj_counts, cell_incl_status))

In [ ]:
sj_counts = pd.concat(pseudo_list, axis=1, keys=[f"Sample{i+1}" for i in range(len(pseudo_list))])
non_zero_count = (sj_counts.sum(axis=1) > 0).values # Subset to rows with > 0 counts
sj_counts_filtered = sj_counts.loc[non_zero_count]

In [ ]:
outdir = "/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/data"
sj_counts_filtered.to_csv(f"{outdir}/sj_counts.csv")